In [1]:
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime
from pathlib import Path
import os
from os.path import basename
import collections
import tflearn
from collections import deque
from sklearn.model_selection import train_test_split
from tflearn.data_utils import to_categorical
import numpy as np


In [2]:
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def check_files(folder):
    """
    Given path to folder, returns whether it's empty or not
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    if(len(filenames)==0):
        return False
    return True

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(label2id, label):
    """
    Returns label for a folder
    """
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)
    return X

def get_image_map(folder, label):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    image_map = {}
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = np.load(f)
        img_arr = img_arr.flatten()
        name = basename(f)
        num = int(name.split("_")[1].split(".")[0])
        image_map[num] = [img_arr, label]

    return image_map

def get_train_data(data_root_path, label_mapping_path):
    """
    Return X and y
    """
    X = []
    y = []
    labels = os.listdir(data_root_path)
    id2label, label2id = get_label_mapping(label_mapping_path)
    print(label2id)
    image_map = {}
    for label in labels:
        train_data_path = data_root_path + label
        if(check_files(train_data_path)) :
            temp_map = get_image_map(train_data_path, label)
            image_map.update(temp_map)
    new_map = collections.OrderedDict(sorted(image_map.items()))
    for k, v in new_map.items():
        X.append(v[0])
        y.append(get_label(label2id, v[1]))
    X = np.array(X)
    return X, np.array(y)

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
def get_data_with_seq(X_batch, y_batch, seq_length, num_classes, input_length):
    """
    creates data where every data point has seq_length images
    """
    X = []
    y = []
    
    image_seq = deque()
    for row in range(0,len(X_batch)):
        image_seq.append(X_batch[row])
        #y_list.append(y_from_image[row])
        if len(image_seq) == seq_length:
            X.append(np.array(list(image_seq)))
            y.append(y_batch[row])
            #y.append(np.array(list(y_list)))
            image_seq.popleft()
            #y_list.popleft()
    
    X = np.array(X)
    y = np.array(y)
    #print(X.shape)
    y = to_categorical(y, num_classes)
    #print(y.shape)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify = y)

    return X_train, X_test, y_train, y_test
    

In [5]:
def get_data(root_path, label_mapping_path):
    folders = os.listdir(root_path)
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    input_length = 128
    seq_length = 2
    num_classes = 3
    
    for folder in folders:
        data_root_path = root_path+folder+'/'
        X_batch, y_batch = get_train_data(data_root_path, label_mapping_path)
        X_train_batch, X_test_batch, y_train_batch, y_test_batch = get_data_with_seq(X_batch, y_batch, seq_length, num_classes, input_length)
        X_train.extend(X_train_batch)
        y_train.extend(y_train_batch)
        X_test.extend(X_test_batch)
        y_test.extend(y_test_batch)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    return X_train, X_test, y_train, y_test
        

In [6]:
def get_rnn(images, input_size, num_classes):
    rnn = tflearn.input_data(shape=[None, images, input_size])
    rnn = tflearn.lstm(rnn, 512, dropout=0.8, return_seq=True)
    rnn = tflearn.lstm(rnn, 512)
    rnn = tflearn.fully_connected(rnn, num_classes, activation='softmax')
    rnn = tflearn.regression(rnn, optimizer='adam',
                             loss='categorical_crossentropy', name="output1")
    return rnn

In [7]:
def train(X_train, y_train, X_test, y_test, seq_length, batch_size):
    
    
    input_length = X_train.shape[2]
    num_classes = len(y_train[0])
    
    rnn = get_rnn(seq_length, input_length, num_classes)

    model = tflearn.DNN(rnn, tensorboard_verbose=1)
    model.fit(X_train, y_train, validation_set=(X_test, y_test),
              show_metric=True, batch_size=batch_size, snapshot_step=100,
              n_epoch=3)
    
    #builder = tf.saved_model.builder.SavedModelBuilder('./modelPath1')
    #builder.add_meta_graph_and_variables(model.session, [tf.saved_model.tag_constants.TRAINING],
                                         #signature_def_map=None, assets_collection=None)
    model.save('checkpoints/rnn.tflearn')
    
    #test to predict lable for just one image
    #t = []
    #t.append(X_train[0])
    #print(model.predict_label(t))
    results =  model.predict_label(X_test)
    
    #builder.save()
    return results

In [8]:
if __name__ == '__main__':
    input_length = 128
    seq_length = 2
    batch_size = 32
    num_classes = 4
    
    root_path = '/mnt/c/Users/bk262/Desktop/last_layer3/'
    label_mapping_path = './labels.txt' #labels.txt should NOT be in the root_path
    
#     X_from_image, y_from_image = get_train_data(data_root_path, label_mapping_path)
#     X_train, X_test, y_train, y_test = get_data_with_seq(X_from_image, y_from_image, seq_length, num_classes, input_length)
#     print(X_train.shape)
    X_train, X_test, y_train, y_test = get_data(root_path, label_mapping_path)
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    results = train(X_train, y_train, X_test, y_test, seq_length, batch_size)

Training Step: 257  | total loss: 0.31003 | time: 18.582s
| Adam | epoch: 003 | loss: 0.31003 - acc: 0.8824 -- iter: 2720/2740
Training Step: 258  | total loss: 0.29866 | time: 19.919s
| Adam | epoch: 003 | loss: 0.29866 - acc: 0.8911 | val_loss: 0.26723 - val_acc: 0.9184 -- iter: 2740/2740
--
INFO:tensorflow:/root/rnn/checkpoints/rnn.tflearn is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [9]:
print(results.shape)
results = np.array(results)
print(len(results[:,0]))
print(len(y_test))

(686, 3)
686
686


In [10]:
from sklearn.metrics import confusion_matrix
y_true = np.argmax(y_test, axis=1)
y_pred = results[:,0]
cmat = confusion_matrix(y_true, y_pred)
cmat
cmat.diagonal()/cmat.sum(axis=1)

array([ 0.92857143,  0.90833333,  0.95081967])

In [11]:
for k in range(0, 3):
    count = 0
    counti = 0
    for i in range(0, len(y_true)):
        if(y_true[i]==k):
            counti=counti+1
            if(y_pred[i]==k):
                count = count+1
    print(k)
    print(count/counti)

0
0.9285714285714286
1
0.9083333333333333
2
0.9508196721311475
